In [1]:
import pandas as pd
from download.downloaders import download

In [39]:

def process_CV2():

    df = pd.read_csv('data/indicator/CV2/raw/CV2_OCEANHEALTHINDEX.M.csv')
    df = df[(df['goal'] == 'TR') & (df.dimension == 'score')]
    df = df[['scenario', 'value', 'region_name']]
    df = df.rename(columns={
        'scenario': 'Year',
        'value': 'Value',
        'region_name': 'Country'
    })

    return df

test = process_CV2()

In [41]:
test.query('Country == "Germany"')

,Year,Value,Country
23866,2012,91.90,Germany
48232,2013,88.85,Germany
72598,2014,92.30,Germany
96964,2015,92.96,Germany
121330,2016,92.51,Germany
145696,2017,92.88,Germany
170062,2018,93.06,Germany
194428,2019,93.57,Germany
218794,2020,93.57,Germany


In [13]:
import pandas as pd
from processing.utils import add_ISO

def process_GN1():
    df = (
        pd.read_csv('data/indicator/GN1/raw/GN1_WIPO.M.csv')
          .drop(columns=['Origin', 'Office (Code)']).rename(columns={'Office': 'Country'})
          .melt(id_vars=['Country', 'Field of technology'], var_name='Year', value_name='Value')
    )
    
    df = add_ISO(df)
    
    env = df[df['Field of technology'] == '24 - Environmental technology'].drop(columns=['Field of technology']).set_index(['Country', 'ISO', 'Year'])
    total = df.groupby(['Country', 'ISO', 'Year']).sum()
    
    df = (env / total).reset_index()
    df['Value'] = df.groupby(['ISO'])['Value'].transform(lambda x: x.rolling(7, 1).mean())

    return df

In [15]:
df = process_GN1()
df.query('ISO == "FRA"')

,Country,ISO,Year,Value
1120,France,FRA,1980,0.014346
1121,France,FRA,1981,0.013340
1122,France,FRA,1982,0.012924
1123,France,FRA,1983,0.012545
1124,France,FRA,1984,0.012072
1125,France,FRA,1985,0.012032
1126,France,FRA,1986,0.011996
1127,France,FRA,1987,0.011627
1128,France,FRA,1988,0.011449
1129,France,FRA,1989,0.011587


In [ ]:
df = process_GN1()

In [ ]:
df.query('ISO == "DEU"').plot(kind='line', x='Year', y='Value')

In [ ]:
df

In [ ]:
df = pd.read_csv('data/indicator/GE3/preprocessed/GE3.1_CW.csv')

In [ ]:
test = pd.read_csv('data/full_data/result.csv')

In [ ]:
test.query("Year == 2020 and Aggregation == 'Index'")

In [ ]:
import json
with open('data/indicator/GE3/raw/GE3.1_CW.json', 'r') as outfile:
    data = json.load(outfile)


In [ ]:
config_3 = {
            "GGI_code": "GE3.1",
            "params": {
                "gas_ids[]": [
                    327,
                    328,
                    329
                ],
                "sector_ids[]": [
                    1304,
                    1302
                ],
                "source_ids[]": [111]
            }
        }

In [ ]:
data_json = download('CW', config_3, path=None, restart=False)

In [ ]:
data_json

In [ ]:
from processing.api_preprocessors import CW_Preprocessor, preprocess_raw_dict

In [ ]:
df = preprocess_raw_dict(data_json, CW_Preprocessor)

In [ ]:
df

In [ ]:
test = pd.json_normalize(data_json['data'])

In [ ]:
test

In [ ]:
import requests

def get_CW_API_ids(query_param, API_URL='https://www.climatewatchdata.org/'):
    """

    """            
    url     = f'{API_URL}/api/v1/data/historical_emissions/{query_param}'

    response = requests.get(url, params=query_param, headers={"Accept": "application/json"})
    
    
    data_json = response.json()
    df = pd.json_normalize(data_json['data']) 
    
    return df

In [ ]:
test = get_CW_API_ids(query_param="sectors")


In [ ]:
test

In [ ]:
gases